In [116]:
import pandas as pd
import PyPDF2 as p2
import re
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os
from PyPDF2 import PdfFileReader, PdfFileWriter

from PIL import Image
# import ghostscript
# from pdf2image import convert_from_path

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [117]:
#import location_extractor
import re 

import re
#import math
#import collections

#import gis_converter
import html_analyzer
import util

In [118]:
tables_df = pd.read_csv("F:\\Environmental Baseline Data\\Version 4 - Final\\Support files\\Table titles raw data\\table_titles.csv")
tables_df.head()

,page_number,table_title,DataID,file_name,application_name,esa_folder_link,DocumentID,esa_download_link,DataID_pdf,Section,...,Consultant Name,Status in GIS,RTS English Title = Project name,Regulatory Instruments Issued,Pipeline Named in GIS,Application title short,Link to Application,Link to Decision,Location,Category
0,5,FIGURE 1 REGIONAL LOCATION,1059614,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),2013-11-08 Application for North Montney Proje...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,1059614.pdf,52,...,"Stantec Consulting Ltd., TERA Environmental Co...",Operating/Approved Kahta Section,NGTL North Montney Project,GC-125,North Montney Mainline,Application for North Montney Project,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,TBD,Figure
1,6,FIGURE 2 LOCATION OF WATERCOURSE CROSSINGS,1059614,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),2013-11-08 Application for North Montney Proje...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,1059614.pdf,52,...,"Stantec Consulting Ltd., TERA Environmental Co...",Operating/Approved Kahta Section,NGTL North Montney Project,GC-125,North Montney Mainline,Application for North Montney Project,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,TBD,Figure
2,8,TABLE 1,1059614,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),2013-11-08 Application for North Montney Proje...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,1059614.pdf,52,...,"Stantec Consulting Ltd., TERA Environmental Co...",Operating/Approved Kahta Section,NGTL North Montney Project,GC-125,North Montney Mainline,Application for North Montney Project,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,TBD,Table
3,9,TABLE 2,1059614,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),2013-11-08 Application for North Montney Proje...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,1059614.pdf,52,...,"Stantec Consulting Ltd., TERA Environmental Co...",Operating/Approved Kahta Section,NGTL North Montney Project,GC-125,North Montney Mainline,Application for North Montney Project,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,TBD,Table
4,14,TABLE 3,1059614,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),2013-11-08 Application for North Montney Proje...,https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,1059614.pdf,52,...,"Stantec Consulting Ltd., TERA Environmental Co...",Operating/Approved Kahta Section,NGTL North Montney Project,GC-125,North Montney Mainline,Application for North Montney Project,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,TBD,Table


In [119]:
list_titles = tables_df['table_title']
len(list_titles)
list_titles[1]

33082

'FIGURE 2 LOCATION OF WATERCOURSE CROSSINGS'

# Categorization Function 

In [121]:
def categorize_tables(list_titles):
    categories = []
    for title in list_titles: 
        
        category = 0
        
        table_string = title
        splts = re.split('\d+[\.:]?\s+',table_string) #one period or none after the last digit followed by one or more whitespace
        splt = []
        for spl in splts:
            if len(spl) > 0:
                splt.append(spl)
        
        title_words = re.split(" ", title)
        
        title_words_no_blanks = []
        for title_word in title_words:
            if len(title_word) > 0:
                title_words_no_blanks.append(title_word)
          
        category = 0
        
        if(len(splt) > 1) and len(splt[1]) > 0:
            #print(splt[1])
            if splt[1][0].isupper() or splt[1][0].isdigit():
                category = 1
                    
        if len(title_words_no_blanks) < 4:
            category = 0.5
            
        categories.append(category)
        print("{} is {}".format(title, category))
            
    return(categories)
    

In [123]:
categories = categorize_tables(list_titles)
#categories

FIGURE 1 REGIONAL LOCATION is 1
FIGURE 2 LOCATION OF WATERCOURSE CROSSINGS is 1
TABLE 1 is 0.5
TABLE 2 is 0.5
TABLE 3 is 0.5
TABLE 4 is 0.5
TABLE 4  Cont'd is 0.5
TABLE 4  Cont'd is 0.5
TABLE 4  Cont'd is 0.5
TABLE 4  Cont'd is 0.5
TABLE 4  Cont'd is 0.5
TABLE 4  Cont'd is 0.5
TABLE 5 is 0.5
TABLE 5  Cont’d is 0.5
TABLE 5  Cont’d is 0.5
TABLE 5  Cont’d is 0.5
TABLE 6 is 0.5
TABLE 6  Cont’d is 0.5
TABLE 6  Cont’d is 0.5
TABLE 6  Cont’d is 0.5
TABLE 6  Cont’d is 0.5
TABLE 7 is 0.5
TABLE 8 is 0.5
TABLE 8  Cont'd is 0.5
Table 3-1 Wildlife Species of Management Concern in the RAA is 1
Table 3-1 Wildlife Species of Management Concern in the RAA is 1
Table 3-2 EOSD Vegetation Cover Classes and Project Definitions is 1
Table 3-3 summarizes species-habitat associations from the Land Cover Circa 2000 Vector data used in is 0
Table 3-3 Species-Habitat Associations Used to Determine Availability of Suitable Habitat is 1
Table 3-3 Species-Habitat Associations Used to Determine Availability of Suita

TABLE 3.8 is 0.5
Table 3.9 lists the consumption rates assumed for Aboriginal peoples living within the SLHHRA LSAs for  the Sumas Terminal, Burnaby Terminal and Westridge Marine Terminal. is 0
TABLE 3.9 is 0.5
TABLE 3.10 is 0.5
TABLE 3.11 is 0.5
Table 3.12 summarizes the relevant physical-chemical properties and fugacity model results for each of  the chemicals potentially emitted from the Project, and identifies those chemicals included in the multiple  pathway assessment. is 0
TABLE 3.12 is 0.5
TABLE 3.13 is 0.5
TABLE 3.14 is 0.5
TABLE 3.15 is 0.5
TABLE 3.16 is 0.5
TABLE 3.17 is 0.5
TABLE 3.18 is 0.5
TABLE 3.19 is 0.5
TABLE 3.20 is 0.5
TABLE 3.21 is 0.5
TABLE 3.22 is 0.5
Table 4.1 presents region-specific health statistics for health endpoints considered to be particularly  relevant for assessing the potential health effects that can result from chemical exposures associated with  Project-related emissions from the tank terminals. These endpoints include certain cancers and other  c

TABLE 3.3-3  Cont'd is 0.5
TABLE 3.3-4 is 0.5
TABLE 3.3-4  Cont'd is 0.5
TABLE 3.3-5 is 0.5
TABLE 3.3-5  Cont'd is 0.5
FIGURE 5.4-4 HUMAN OCCUPANCY  AND RESOURCE USE is 1
FIGURE 1.1 PROJECT OVERVIEW is 1
Table 1. Summary of Consultation Activities Related to Soil. is 1
Table 2.     Extent of Various Soils Along the Proposed Corridor – Edmonton to Edson (RK247.3)      dl(Concluded) is 1
Table 4. Suitability Ratings of the Soils for Reclamation – Edmonton to Edson (RK247.3). is 1
Table 4. Suitability Ratings of the Soils for Reclamation – Edmonton to Edson (RK247.3) (Concluded). is 1
Table 5.        Evaluation of the Soils for Alternate Soil Handling – Edmonton to Edson (RK247.3) is 1
Table 5.      Evaluation of the Soils for Alternate Soil Handling – Edmonton to Edson (RK247.3) (Concluded) is 1
Table 6.      Wind and Water Soil Erosion Hazard Ratings of Map Units Along the Proposed Corridor -   Edmonton to Edson (RK247.3)*. is 1
Table 6.      Wind and Water Soil Erosion Hazard Ratings o

TABLE B-20  Cont'd is 0.5
TABLE B-20  Cont'd is 0.5
TABLE B-20  Cont'd is 0.5
TABLE B-20  Cont'd is 0.5
TABLE B-20  Cont'd is 0.5
TABLE B-20  Cont'd is 0.5
TABLE B-21 is 0.5
TABLE B-21  Cont'd is 0.5
TABLE B-22 is 0.5
TABLE B-22  Cont'd is 0.5
TABLE C-1 is 0.5
TABLE C-1 Cont'd is 0.5
TABLE C-1 Cont'd is 0.5
TABLE C-3 is 0.5
TABLE C-3  Cont’d is 0.5
TABLE C-4 is 0.5
TABLE C-4  Cont'd is 0.5
TABLE C-4  Cont'd is 0.5
TABLE C-4  Cont'd is 0.5
TABLE C-5 is 0.5
TABLE C-5  Cont'd is 0.5
TABLE C-6 is 0.5
TABLE C-6  Cont'd is 0.5
TABLE C-6  Cont'd is 0.5
TABLE C-7 is 0.5
TABLE C-7  Cont'd is 0.5
TABLE C-8 is 0.5
TABLE C-8  Cont'd is 0.5
TABLE C-10 is 0.5
TABLE C-11 is 0.5
TABLE C-12 is 0.5
TABLE C-13 is 0.5
TABLE C-14 is 0.5
TABLE D-1 is 0.5
TABLE D-1  Cont’d is 0.5
TABLE D-2 is 0.5
TABLE D-2  Cont'd is 0.5
TABLE D-2  Cont'd is 0.5
TABLE D-2  Cont'd is 0.5
TABLE D-2  Cont'd is 0.5
TABLE D-2  Cont'd is 0.5
TABLE D-3 is 0.5
TABLE D-3  Cont'd is 0.5
TABLE D-3  Cont'd is 0.5
TABLE D-4 is 0.5
TABLE 

TABLE I-2  Cont'd is 0.5
TABLE I-2  Cont'd is 0.5
TABLE I-2  Cont'd is 0.5
TABLE I-2  Cont'd is 0.5
TABLE I-2  Cont'd is 0.5
TABLE I-2  Cont'd is 0.5
TABLE I-2  Cont'd is 0.5
TABLE I-2  Cont'd is 0.5
TABLE I-2  Cont'd is 0.5
TABLE I-2  Cont'd is 0.5
TABLE I-2  Cont'd is 0.5
TABLE I-2  Cont'd is 0.5
TABLE I-2  Cont'd is 0.5
TABLE I-2  Cont'd is 0.5
TABLE I-2  Cont'd is 0.5
TABLE I-2  Cont'd is 0.5
TABLE I-2  Cont'd is 0.5
TABLE I-2  Cont'd is 0.5
TABLE I-2  Cont'd is 0.5
TABLE I-2  Cont'd is 0.5
TABLE I-2  Cont'd is 0.5
TABLE I-2  Cont'd is 0.5
TABLE I-3 is 0.5
TABLE I-3  Cont'd is 0.5
TABLE J-1 is 0.5
TABLE J-1  Cont'd is 0.5
TABLE J-1  Cont'd is 0.5
TABLE J-1  Cont'd is 0.5
TABLE J-1  Cont'd is 0.5
TABLE J-2 is 0.5
TABLE K-1 is 0.5
TABLE K-1  Cont'd is 0.5
TABLE K-1  Cont'd is 0.5
TABLE K-1  Cont'd is 0.5
TABLE K-1  Cont'd is 0.5
TABLE K-1  Cont'd is 0.5
TABLE K-1  Cont'd is 0.5
TABLE K-1  Cont'd is 0.5
TABLE K-1  Cont'd is 0.5
TABLE K-1  Cont'd is 0.5
TABLE K-1  Cont'd is 0.5
TABLE K

...

TABLE 10  Cont'd is 0.5
FIGURE 4A is 0.5
FIGURE 4B is 0.5
FIGURE 4C is 0.5
TABLE 11 is 0.5
TABLE 12 is 0.5
TABLE 13 is 0.5
TABLE 13  Cont'd is 0.5
FIGURE 5 is 0.5
TABLE 14 is 0.5
Table 15 identifies the established recreational facilities and activities located in the RSA in BC, including  the location of campsites, and Figure 6 identifies their location. is 0
TABLE 15 is 0.5
TABLE 15  Cont'd is 0.5
FIGURE 6 is 0.5
Table 16 identifies the outdoor recreational facilities and activities within the RSA in Alberta, including  campsites and Figure 6 identifies the major recreational sites and trails. is 0
TABLE 16 is 0.5
TABLE 17 is 0.5
TABLE 17  Cont'd is 0.5
Table 18 summarizes the economic effects of the construction of the Project, including direct and indirect  employment and labour income expected in BC and Alberta. Table 19 presents a summary of expected  workforce requirements during construction. The expenditures are estimated at $8.5 million in BC and  $92.7 million in Alberta. Th

Figure 19-1a and Figure 19-1b. For more detailed information on the watercourse crossings, see is 0
FIGURE NO.19-1a is 0.5
FIGURE NO.19-1b Area is 0.5
Table 19-1 Watercourse Crossing Inventory is 1
Figure 19-1b). In most cases, the operating season for the stream-flow stations was from March to is 0
Table 19-2 Water Survey of Canada Stations – Selected for the Analysis is 1
Table 19-3 Study-Specific Streamflow Measurements is 1
Figure 19-3. Based on that comparison, the mean discharges to Skull Creek and Bone Creek are is 1
Figure 19 - 2 Discharge Quantiles and 2008 Hydrograph is 0
Figure 19 - 3 Discharge Quantiles and 2008 Hydrograph is 0
Table 19-4 Index Stations and Transfer Factors Used to Characterize Major Water Crossings is 1
Table 19-5 Estimated March to October Mean Discharges at the Major Crossings is 1
Figure 19 - 4 Estimated Discharge Quantiles is 0
Figure 19 - 5 Estimated Discharge Quantiles is 0
Figure 19 - 6 Estimated Discharge Quantiles is 0
Figure 19 - 7 Discharge Quan

Table 6-24 Water Erosion Risk for Non-agricultural Soils is 1
Table 6-24 Water Erosion Risk for Non-agricultural Soils (cont’d) is 1
Table 6-25 Characterization of the Residual Effect of Effects on Soil Quality from Physical Disturbance is 1
Table 6-25 Characterization of the Residual Effect of Effects on Soil Quality from Physical Disturbance (cont’d) is 1
Table 6-25 Characterization of the Residual Effect of Effects on Soil Quality from Physical Disturbance (cont’d) is 1
Table 6-25 Characterization of the Residual Effect of Effects on Soil Quality from Physical Disturbance (cont’d) is 1
Table 6-25 Characterization of the Residual Effect of Effects on Soil Quality from Physical Disturbance (cont’d) is 1
Table 6-25 Characterization of the Residual Effect of Effects on Soil Quality from Physical Disturbance (cont’d) is 1
Table 6-26 Baseline Regional Soil Acid Sensitivity Ratings is 1
Figure 6-3. The area within the security fence is excluded from the assessment and is classified as is 1

Tables 3.2 and 3.3 as well as Appendices 7A and 7B of this ESA provides a summary of the meetings  and interviews that took place for the socio-economic component of the ESA. The issues that were raised  and where they are considered in the socio-economic assessment are also summarized in these tables  and appendices. is 0
TABLE 3.2 is 0.5
TABLE 3.2  Cont'd is 0.5
TABLE 3.2  Cont'd is 0.5
TABLE 3.2  Cont'd is 0.5
TABLE 3.2  Cont'd is 0.5
TABLE 3.2  Cont'd is 0.5
TABLE 3.2  Cont'd is 0.5
TABLE 3.2  Cont'd is 0.5
TABLE 3.3 is 0.5
TABLE 3.3  Cont’d is 0.5
TABLE 3.3  Cont’d is 0.5
TABLE 3.3  Cont’d is 0.5
TABLE 3.3  Cont’d is 0.5
TABLE 3.3  Cont’d is 0.5
TABLE 3.3  Cont’d is 0.5
TABLE 3.3  Cont’d is 0.5
TABLE 3.3  Cont’d is 0.5
TABLE 3.3  Cont’d is 0.5
TABLE 3.3  Cont’d is 0.5
TABLE 3.3  Cont’d is 0.5
TABLE 3.3  Cont’d is 0.5
TABLE 3.3  Cont’d is 0.5
TABLE 3.3  Cont’d is 0.5
TABLE 3.3  Cont’d is 0.5
TABLE 3.3  Cont’d is 0.5
TABLE 3.3  Cont’d is 0.5
TABLE 3.3  Cont’d is 0.5
TABLE 3.3  Cont’

Table 6.35 provides a summary of the significance evaluation of potential residual socio-economic effects  of the construction and operation of the proposed pipeline on human health. The rationale used to  evaluate the significance of each of the potential residual socio-economic effects is provided below. is 0
TABLE 6.35 is 0.5
TABLE 6.36 is 0.5
TABLE 6.36  Cont'd is 0.5
Table 6.37 provides a summary of the significance evaluation of potential residual socio-economic effects  of the construction and operation of the proposed pipeline on infrastructure and services. The rationale  used to evaluate the significance of each of the residual socio-economic effects is provided below. An  evaluation of significance is not required for those potential effects where no residual effect is identified  (i.e., provision of emergency services during operations). is 0
TABLE 6.37 is 0.5
TABLE 6.38 is 0.5
Table 6.39 provides a summary of the significance evaluation of potential residual socio-economic

# TEST - 2

In [124]:
index2 = pd.read_csv('F:\Environmental Baseline Data\Version 3\Indices\Final\Index 2 - List of PDFs for Major Projects with ESAs.csv')
ngtl_2021 = index2[index2['Application title short'].str.contains("Application for 2021 NGTL System Expansion Project")]

In [125]:
index2 = pd.read_csv('F:\Environmental Baseline Data\Version 3\Indices\Final\Index 2 - List of PDFs for Major Projects with ESAs.csv')
ngtl_2021 = index2[index2['Application title short'].str.contains("Application for 2021 NGTL System Expansion Project")]
ngtl_2021_pdfs = ngtl_2021['PDF file name'].tolist()

path = 'F:/Environmental Baseline Data/Version 3/Data/PDF/'
ngtl_2021_pdfs_path = [path + x for x in ngtl_2021_pdfs]


def get_list_elements(s, level):
    if isinstance(s, list):
        for x in s:
            get_list_elements(x, level+1)
    else:
        data.append({'pdf_name': y.split('/')[-1], 'outline_item' : s.title, 'page_number' : pdf_reader.getDestinationPageNumber(s) + 1, 'Level' : level, 'pdf_path' : y})
    return

data = []
for y in ngtl_2021_pdfs_path:
    PDFfile = open(y, 'rb')
    pdf_reader = p2.PdfFileReader(PDFfile)
    
    get_list_elements(pdf_reader.outlines, 0)
df_outlines = pd.DataFrame(data)

In [126]:

#LABEL_HEADINGS =        ('plate', 'figure', 'photo', 'image', 'table', 'appendix')
#LABEL_HEADINGS_PLURAL = ('plates', 'figures', 'photos', 'images', 'tables', 'appendices')
searchlist = ['Image', 'Figure', 'Photo']
searchfor = []
for search in searchlist:
    searchfor.append(search)
    searchfor.append(search + 's')
    searchfor.append(search.upper())
    searchfor.append(search.upper() + 'S')
searchfor

['Image',
 'Images',
 'IMAGE',
 'IMAGES',
 'Figure',
 'Figures',
 'FIGURE',
 'FIGURES',
 'Photo',
 'Photos',
 'PHOTO',
 'PHOTOS']

In [127]:
df_outlines_figures = df_outlines[df_outlines['outline_item'].str.contains('|'.join(searchfor), na = False)] 
df_outlines_figures.head(10)

,pdf_name,outline_item,page_number,Level,pdf_path
8,A6F4Q3.pdf,Figure 1.3–1: 2021 NGTL System Expansion Proje...,16,5,F:/Environmental Baseline Data/Version 3/Data/...
46,A6F4Q3.pdf,Figure 2.6–1: Preliminary Construction Schedule,50,5,F:/Environmental Baseline Data/Version 3/Data/...
68,A6F4Q3.pdf,Figure 4.3–1: Local Study Area (Pipeline),71,4,F:/Environmental Baseline Data/Version 3/Data/...
69,A6F4Q3.pdf,Figure 4.3–2: Local Study Area (Compressor Sta...,72,4,F:/Environmental Baseline Data/Version 3/Data/...
70,A6F4Q3.pdf,Figure 4.3–3: Regional Study Area,73,4,F:/Environmental Baseline Data/Version 3/Data/...
87,A6F4Q3.pdf,Figure 4.4–1: Study Areas for Cumulative Effects,90,4,F:/Environmental Baseline Data/Version 3/Data/...
92,A6F4Q3.pdf,Figure 4.4–2: Forestry Management Areas,99,4,F:/Environmental Baseline Data/Version 3/Data/...
95,A6F4Q3.pdf,Figure 4.4–3: Population Projections Scenario ...,101,4,F:/Environmental Baseline Data/Version 3/Data/...
98,A6F4Q3.pdf,Figure 4.4–4: Reasonably Foreseeable Projects,104,4,F:/Environmental Baseline Data/Version 3/Data/...
112,A6F4Q4.pdf,Figure 5.1–1: Natural Regions and Subregions,5,4,F:/Environmental Baseline Data/Version 3/Data/...


In [128]:
df_outlines_figures.iloc[2]['outline_item']
len(df_outlines_figures)
df_outlines_figures.iloc[2]['pdf_path']

'Figure 4.3–1: Local Study Area (Pipeline)'

388

'F:/Environmental Baseline Data/Version 3/Data/PDF/A6F4Q3.pdf'

In [129]:
categories = categorize_tables(df_outlines_figures['outline_item'])
categories

Figure 1.3–1: 2021 NGTL System Expansion Project Overview is 1
Figure 2.6–1: Preliminary Construction Schedule is 1
Figure 4.3–1: Local Study Area (Pipeline) is 1
Figure 4.3–2: Local Study Area (Compressor Station Unit Additions) is 1
Figure 4.3–3: Regional Study Area is 1
Figure 4.4–1: Study Areas for Cumulative Effects  is 1
Figure 4.4–2: Forestry Management Areas is 1
Figure 4.4–3: Population Projections Scenario in the SESA is 1
Figure 4.4–4: Reasonably Foreseeable Projects is 1
Figure 5.1–1: Natural Regions and Subregions is 1
Figure 6.1–1: Elmworth Section 2017 Field Data Collection is 1
Figure 6.1–2: Soil Orders Valhalla Section is 1
Figure 6.1–3: Soil Orders Elmworth Section is 1
Figure 6.1–4: Soil Orders Karr Section is 1
Figure 6.1–5: Soil Orders Karr and Deep Valley Sections is 1
Figure 6.1–6:  Soil Orders Deep Valley Section is 1
Figure 6.1–7: Soil Orders Deep Valley and McLeod River Connection Sections is 1
Figure 6.1–8: Soil Orders Robb Section is 1
Figure 6.1–9: Soil Ord

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0.5,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0.5,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [130]:
splt = re.split('\d+[\.:]?\s+',df_outlines_figures.iloc[0]['outline_item'])
splt

['Figure 1.3–', '', 'NGTL System Expansion Project Overview']